In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import rpreactor

rpreactor.__version__

'0.0.12+61.g12dad48.dirty'

# Errors and special cases

When something goes wrong, we prefer to log the event rather to yield an Exception and terminate. Consequently you will find useful to configure the logs:

In [3]:
import logging

logging.basicConfig(level=logging.INFO)

Some example molecule and rule:

In [4]:
inchi = "InChI=1S/C12H22O11/c13-1-3-5(15)6(16)9(19)12(22-3)23-10-4(2-14)21-11(20)8(18)7(10)17/h3-20H,1-2H2/t3-,4-,5+,6+,7-,8-,9-,10-,11?,12+/m1/s1"
rsmarts = "([#6@@&v4:1]1(-[#8&v2:2]-[#1&v1:3])(-[#1&v1:4])-[#6@&v4:5](-[#8&v2:6]-[#1&v1:7])(-[#1&v1:8])-[#6@&v4:9](-[#8&v2:10]-[#1&v1:11])(-[#1&v1:12])-[#6@&v4:13](-[#6&v4:14](-[#8&v2:15]-[#1&v1:16])(-[#1&v1:17])-[#1&v1:18])(-[#1&v1:19])-[#8&v2:20]-[#6@&v4:21]-1(-[#8&v2:22]-[#6@@&v4:23]1(-[#1&v1:24])-[#6@@&v4:25](-[#8&v2:26]-[#1&v1:27])(-[#1&v1:28])-[#6@&v4:29](-[#8&v2:30]-[#1&v1:31])(-[#1&v1:32])-[#6&v4:33](-[#8&v2:34]-[#1&v1:35])(-[#1&v1:36])-[#8&v2:37]-[#6@@&v4:38]-1(-[#6&v4:39](-[#8&v2:40]-[#1&v1:41])(-[#1&v1:42])-[#1&v1:43])-[#1&v1:44])-[#1&v1:45])>>([#6@@&v4:1]1(-[#8&v2:2]-[#1&v1:3])(-[#1&v1:4])-[#6@&v4:5](-[#8&v2:6]-[#1&v1:7])(-[#1&v1:8])-[#6@&v4:9](-[#8&v2:10]-[#1&v1:11])(-[#1&v1:12])-[#6@&v4:13](-[#6&v4:14](-[#8&v2:15]-[#1&v1:16])(-[#1&v1:17])-[#1&v1:18])(-[#1&v1:19])-[#8&v2:20]-[#6@&v4:21]-1(-[#8&v2:22]-[#1&v1])-[#1&v1:45].[#6@@&v4:29]1(-[#8&v2:30]-[#1&v1:31])(-[#1&v1:32])-[#6@&v4:25](-[#8&v2:26]-[#1&v1:27])(-[#1&v1:28])-[#6@@&v4:23](-[#8&v2]-[#1&v1])(-[#1&v1:24])-[#6@&v4:38](-[#6&v4:39](-[#8&v2:40]-[#1&v1:41])(-[#1&v1:42])-[#1&v1:43])(-[#1&v1:44])-[#8&v2:37]-[#6&v4:33]-1(-[#8&v2:34]-[#1&v1:35])-[#1&v1:36])"

## Insert error

Identifiers MUST be unique to their catagory (chemicals or rules) in the database. Molecules and rules can be duplicated as long as the identifier is different. Any attempt to insert a record with an identifier already registred will yield an IntegrityError exception.

In [5]:
import sqlite3


o = rpreactor.RuleBurner(with_hs=True)

try:
    o.insert_inchi({'one': inchi, 'two': inchi, 'three': inchi})  # this gets committed
    o.insert_inchi({'four': inchi, 'one': inchi})                 # this will be lost
    o.insert_inchi({'six': inchi})                                # this too
except sqlite3.IntegrityError as err:
    print(err)

o.chemicals  # list of currently used identifiers

INFO:rpreactor.rule.burner:Connected to a database with 0 rules, 0 molecules, and 0 results (at ':memory:').


UNIQUE constraint failed: molecules.id


['four', 'one', 'three', 'two']

Identifiers chosen by rpreactor are safe:

In [6]:
o = rpreactor.RuleBurner(with_hs=True)
o.insert_inchi([inchi, inchi, inchi])
o.insert_inchi({3: inchi})  # manually steal the identifier 3
o.insert_inchi([inchi])

o.chemicals  # list of currently used identifiers

INFO:rpreactor.rule.burner:Connected to a database with 0 rules, 0 molecules, and 0 results (at ':memory:').


['0', '1', '2', '3', '4']

## Errors due to invalid input

Invild input gets logged but does not yield any exception.

In [7]:
bad_rsmarts = "([#6@@&v4:1]1(-[#8&dgd])-[#6@&v4:5](-[#8&v2:6]-[#1&v1:7])(-[#1&v1:8])-[#6@&v4:9](-[#8&v2:10]-[#1&v1:11])(-[#1&v1:12])-[#6@&v4:13](-[#6&v4:14](-[#8&v2:15]-[#1&v1:16])(-[#1&v1:17])-[#1&v1:18])(-[#1&v1:19])-[#8&v2:20]-[#6@&v4:21]-1(-[#8&v2:22]-[#6@@&v4:23]1(-[#1&v1:24])-[#6@@&v4:25](-[#8&v2:26]-[#1&v1:27])(-[#1&v1:28])-[#6@&v4:29](-[#8&v2:30]-[#1&v1:31])(-[#1&v1:32])-[#6&v4:33](-[#8&v2:34]-[#1&v1:35])(-[#1&v1:36])-[#8&v2:37]-[#6@@&v4:38]-1(-[#6&v4:39](-[#8&v2:40]-[#1&v1:41])(-[#1&v1:42])-[#1&v1:43])-[#1&v1:44])-[#1&v1:45])>>([#6@@&v4:1]1(-[#8&v2:2]-[#1&v1:3])(-[#1&v1:4])-[#6@&v4:5](-[#8&v2:6]-[#1&v1:7])(-[#1&v1:8])-[#6@&v4:9](-[#8&v2:10]-[#1&v1:11])(-[#1&v1:12])-[#6@&v4:13](-[#6&v4:14](-[#8&v2:15]-[#1&v1:16])(-[#1&v1:17])-[#1&v1:18])(-[#1&v1:19])-[#8&v2:20]-[#6@&v4:21]-1(-[#8&v2:22]-[#1&v1])-[#1&v1:45].[#6@@&v4:29]1(-[#8&v2:30]-[#1&v1:31])(-[#1&v1:32])-[#6@&v4:25](-[#8&v2:26]-[#1&v1:27])(-[#1&v1:28])-[#6@@&v4:23](-[#8&v2]-[#1&v1])(-[#1&v1:24])-[#6@&v4:38](-[#6&v4:39](-[#8&v2:40]-[#1&v1:41])(-[#1&v1:42])-[#1&v1:43])(-[#1&v1:44])-[#8&v2:37]-[#6&v4:33]-1(-[#8&v2:34]-[#1&v1:35])-[#1&v1:36])"

o = rpreactor.RuleBurner(with_hs=True)
o.insert_inchi([inchi])
o.insert_rsmarts([bad_rsmarts])

results = [x for x in o.compute(rule_mol='*')]
results

INFO:rpreactor.rule.burner:Connected to a database with 0 rules, 0 molecules, and 0 results (at ':memory:').
ERROR:rpreactor.rule.burner:Something went wrong converting rule '0': RULE-CONVERSION-ERROR: ChemicalReactionParserException: Problems constructing reactant from SMARTS: [#6@@&v4:1]1(-[#8&dgd])-[#6@&v4:5](-[#8&v2:6]-[#1&v1:7])(-[#1&v1:8])-[#6@&v4:9](-[#8&v2:10]-[#1&v1:11])(-[#1&v1:12])-[#6@&v4:13](-[#6&v4:14](-[#8&v2:15]-[#1&v1:16])(-[#1&v1:17])-[#1&v1:18])(-[#1&v1:19])-[#8&v2:20]-[#6@&v4:21]-1(-[#8&v2:22]-[#6@@&v4:23]1(-[#1&v1:24])-[#6@@&v4:25](-[#8&v2:26]-[#1&v1:27])(-[#1&v1:28])-[#6@&v4:29](-[#8&v2:30]-[#1&v1:31])(-[#1&v1:32])-[#6&v4:33](-[#8&v2:34]-[#1&v1:35])(-[#1&v1:36])-[#8&v2:37]-[#6@@&v4:38]-1(-[#6&v4:39](-[#8&v2:40]-[#1&v1:41])(-[#1&v1:42])-[#1&v1:43])-[#1&v1:44])-[#1&v1:45]


[]

In [8]:
bad_inchi = "InChI=1S/C12H22O11/c13-1-3-5bdfbf)9(19)12(22-3)23-10-4(2-14)21-11(20)8(18)7(10)17/h3-20H,1-2H2/t3-,4-,5+,6+,7-,8-,9-,10-,11?,12+/m1/s1"

o = rpreactor.RuleBurner(with_hs=True)
o.insert_inchi([bad_inchi])
o.insert_rsmarts([rsmarts])

results = [x for x in o.compute(rule_mol='*')]
results

INFO:rpreactor.rule.burner:Connected to a database with 0 rules, 0 molecules, and 0 results (at ':memory:').
ERROR:rpreactor.rule.burner:Something went wrong converting chemical '0': CHEM-CONVERSION-ERROR: Python argument types in
    rdkit.Chem.rdmolops.Cleanup(NoneType)
did not match C++ signature:
    Cleanup(RDKit::ROMol {lvalue} mol)


[]

## Timeout error

In [9]:
timeout_rsmarts="([#6&v4:1](=[#6&v4:2](-[#6&v4:3])-[#6&v4:4](-[#6&v4:5](-[#6&v4:6](=[#6&v4:7](-[#6&v4:8])-[#6&v4:9](-[#6&v4:10](-[#6&v4:11](=[#6&v4:12](-[#6&v4:13])-[#6&v4:14](-[#6&v4:15](-[#6&v4:16](=[#6&v4:17](-[#6&v4:18])-[#6&v4:19](-[#6&v4:20](-[#6&v4:21](=[#6&v4:22](-[#6&v4:23])-[#6&v4:24](-[#6&v4:25](-[#6&v4:26](=[#6&v4:27](-[#6&v4:28])-[#6&v4:29](-[#6&v4:30](-[#6&v4:31](=[#6&v4:32])-[#1&v1:33])(-[#1&v1:34])-[#1&v1:35])(-[#1&v1:36])-[#1&v1:37])-[#1&v1:38])(-[#1&v1:39])-[#1&v1:40])(-[#1&v1:41])-[#1&v1:42])-[#1&v1:43])(-[#1&v1:44])-[#1&v1:45])(-[#1&v1:46])-[#1&v1:47])-[#1&v1:48])(-[#1&v1:49])-[#1&v1:50])(-[#1&v1:51])-[#1&v1:52])-[#1&v1:53])(-[#1&v1:54])-[#1&v1:55])(-[#1&v1:56])-[#1&v1:57])-[#1&v1:58])(-[#1&v1:59])-[#1&v1:60])(-[#1&v1:61])-[#1&v1:62])(-[#6&v4:63](-[#6&v4:64](-[#6&v4:65](=[#6&v4:66](-[#6&v4:67](-[#6&v4:68](-[#6&v4:69](=[#6&v4:70](-[#6&v4:71](-[#6&v4:72](-[#6&v4:73](=[#6&v4:74](-[#6&v4:75](-[#6&v4:76](-[#6&v4:77](=[#6&v4:78](-[#6&v4:79](-[#6&v4:80](-[#6&v4:81](=[#6&v4:82](-[#6&v4:83](-[#8&v2:84]-[#15&v5:85])(-[#1&v1:86])-[#1&v1:87])-[#1&v1:88])-[#6&v4:89])(-[#1&v1:90])-[#1&v1:91])(-[#1&v1:92])-[#1&v1:93])-[#1&v1:94])-[#6&v4:95])(-[#1&v1:96])-[#1&v1:97])(-[#1&v1:98])-[#1&v1:99])-[#1&v1:100])-[#6&v4:101])(-[#1&v1:102])-[#1&v1:103])(-[#1&v1:104])-[#1&v1:105])-[#1&v1:106])-[#6&v4:107])(-[#1&v1:108])-[#1&v1:109])(-[#1&v1:110])-[#1&v1:111])-[#1&v1:112])-[#6&v4:113])(-[#1&v1:114])-[#1&v1:115])(-[#1&v1:116])-[#1&v1:117])-[#1&v1:118])>>([#15&v5:85]-[#8&v2:84]-[#6&v4:83](-[#6&v4:29](-[#6&v4:27](=[#6&v4:26](-[#1&v1:38])-[#1&v1])-[#6&v4:28])(-[#1&v1:36])-[#1&v1:37])(-[#1&v1:86])-[#1&v1:87].[#6&v4:30](-[#6&v4:31](=[#6&v4:32])-[#1&v1:33])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:34])-[#1&v1:35].[#6&v4:82](=[#6&v4:81](-[#6&v4:89])-[#6&v4:80](-[#6&v4:79](-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:92])-[#1&v1:93])(-[#1&v1:90])-[#1&v1:91])(-[#1&v1:88])-[#1&v1].[#6&v4:78](=[#6&v4:77](-[#6&v4:95])-[#6&v4:76](-[#6&v4:75](-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:98])-[#1&v1:99])(-[#1&v1:96])-[#1&v1:97])(-[#1&v1:94])-[#1&v1].[#6&v4:74](=[#6&v4:73](-[#6&v4:101])-[#6&v4:72](-[#6&v4:71](-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:104])-[#1&v1:105])(-[#1&v1:102])-[#1&v1:103])(-[#1&v1:100])-[#1&v1].[#6&v4:70](=[#6&v4:69](-[#6&v4:107])-[#6&v4:68](-[#6&v4:67](-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:110])-[#1&v1:111])(-[#1&v1:108])-[#1&v1:109])(-[#1&v1:106])-[#1&v1].[#6&v4:66](=[#6&v4:65](-[#6&v4:113])-[#6&v4:64](-[#6&v4:63](-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:116])-[#1&v1:117])(-[#1&v1:114])-[#1&v1:115])(-[#1&v1:112])-[#1&v1].[#6&v4:1](=[#6&v4:2](-[#6&v4:3])-[#6&v4:4](-[#6&v4:5](-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:59])-[#1&v1:60])(-[#1&v1:61])-[#1&v1:62])(-[#1&v1:118])-[#1&v1].[#6&v4:6](=[#6&v4:7](-[#6&v4:8])-[#6&v4:9](-[#6&v4:10](-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:54])-[#1&v1:55])(-[#1&v1:56])-[#1&v1:57])(-[#1&v1:58])-[#1&v1].[#6&v4:11](=[#6&v4:12](-[#6&v4:13])-[#6&v4:14](-[#6&v4:15](-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:49])-[#1&v1:50])(-[#1&v1:51])-[#1&v1:52])(-[#1&v1:53])-[#1&v1].[#6&v4:16](=[#6&v4:17](-[#6&v4:18])-[#6&v4:19](-[#6&v4:20](-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:44])-[#1&v1:45])(-[#1&v1:46])-[#1&v1:47])(-[#1&v1:48])-[#1&v1].[#6&v4:21](=[#6&v4:22](-[#6&v4:23])-[#6&v4:24](-[#6&v4:25](-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(-[#8&v2]-[#15&v5](-[#8&v2]-[#1&v1])(=[#8&v2])-[#8&v2]-[#1&v1])=[#8&v2])(-[#1&v1:39])-[#1&v1:40])(-[#1&v1:41])-[#1&v1:42])(-[#1&v1:43])-[#1&v1])"
timeout_inchi="InChI=1S/C70H116O7P2/c1-57(2)29-16-30-58(3)31-17-32-59(4)33-18-34-60(5)35-19-36-61(6)37-20-38-62(7)39-21-40-63(8)41-22-42-64(9)43-23-44-65(10)45-24-46-66(11)47-25-48-67(12)49-26-50-68(13)51-27-52-69(14)53-28-54-70(15)55-56-76-79(74,75)77-78(71,72)73/h29,31,33,35,37,39,41,43,45,47,49,51,53,55H,16-28,30,32,34,36,38,40,42,44,46,48,50,52,54,56H2,1-15H3,(H,74,75)(H2,71,72,73)"

o = rpreactor.RuleBurner(with_hs=True)
o.insert_inchi([timeout_inchi])
o.insert_rsmarts([timeout_rsmarts])

results = [x for x in o.compute(rule_mol='*', timeout=0.1)]
results

INFO:rpreactor.rule.burner:Connected to a database with 0 rules, 0 molecules, and 0 results (at ':memory:').


[]

In [10]:
print(o)

Connected to a database with 1 rules, 1 compounds, and 0 results (at ':memory:'). Last compute call yield 0 results (0 precomputed, 0 new); 0 errors were caught (details in the logs), and 1 timeouts were hit.


## No match found: the reaction does not apply on this molecule

In [11]:
another_inchi = "InChI=1S/H2O/h1H2"

o = rpreactor.RuleBurner(with_hs=True)

o.insert_inchi([another_inchi])
o.insert_rsmarts([rsmarts])

results = [x for x in o.compute(rule_mol='*')]
results

INFO:rpreactor.rule.burner:Connected to a database with 0 rules, 0 molecules, and 0 results (at ':memory:').


[]

## When the reaction/molecule identifier is not found in the database

In [12]:
o = rpreactor.RuleBurner(with_hs=True)

o.insert_inchi({'inchi1': inchi})
o.insert_rsmarts({'rsmart1': rsmarts})

tasks = [('this_rule_does_not_exist', 'inchi1')]

try:
    result = [x for x in o.compute(tasks)]
except ValueError as err:
    print(err)

INFO:rpreactor.rule.burner:Connected to a database with 0 rules, 0 molecules, and 0 results (at ':memory:').


'this_rule_does_not_exist' is not a valid identifier in task (rid=this_rule_does_not_exist, cid=inchi1).




Notice that you can should use the rulesand chemicals properties to access the list of currently used identifiers:


In [13]:
o.rules

['rsmart1']

In [14]:
o.chemicals

['inchi1']

Those can be handy to check if an identifier is used:

In [15]:
'this_rule_does_not_exist' in o.rules

False